## Loading the data

In [1]:
import numpy as np
import pandas as pd
import ast
import os
import zipfile

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lower, regexp_replace
from pyspark.ml.feature import MinHashLSH, HashingTF
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder \
    .appName("BooksReviewSimilarity") \
    .config("spark.driver.memory", "5g") \
    .master("local[*]") \
    .getOrCreate()

In [5]:
df_cooking = spark.read.csv("df_cooking.csv", header=True, inferSchema=True)

## Preprocessing the Books Data

In [6]:
df_cooking.show(5)

+----------------+--------------------+--------------------+
|           Title|         review/text|             User_id|
+----------------+--------------------+--------------------+
|Alaska Sourdough|I have been using...|       AC58Z72OB2DDX|
|Alaska Sourdough|My poor dogeared,...|      A3CNQIKVTG9QYO|
|Alaska Sourdough|As a former Alask...|      A2UMP9TJTJ6A6B|
|Alaska Sourdough|"For those of us ...| and baking soda ...|
|Alaska Sourdough|Make the most sub...|      A22T74YNRM8NTK|
+----------------+--------------------+--------------------+
only showing top 5 rows



In [7]:
df_cooking.count(), len(df_cooking.columns)

(22434, 3)

In [8]:
# subsample the dataset (50%)
df_cooking = df_cooking.sample(0.5, seed=123)
df_cooking.count(), len(df_cooking.columns)

(11232, 3)

In [9]:
from pyspark.sql.functions import col, upper

# drop rows where User_id is not all caps and that has spaces
df_cooking = df_cooking.filter(
	(col("User_id") == upper(col("User_id"))) & (~col("User_id").contains(" "))
)
df_cooking.count(), len(df_cooking.columns)

(9694, 3)

In [10]:
import re

In [11]:
# ensuring that a user only has one entry per book
df_cooking = df_cooking.dropDuplicates(
    subset=["Title", "User_id"]
)

In [12]:
from pyspark.sql.functions import lower, regexp_replace

In [13]:
df_clean = df_cooking.withColumn(
    "clean_text",
    regexp_replace(lower(col("review/text")), r'[^\w\s]', '')
)

In [14]:
df_clean.show(5)

+--------------------+--------------------+--------------+--------------------+
|               Title|         review/text|       User_id|          clean_text|
+--------------------+--------------------+--------------+--------------------+
|"Cooking with Sei...|The book was deli...|A1FTPHKCHYAJTY|the book was deli...|
|"Cooking with Sei...|Cookbook on Cooki...| AFYO0DI7QNLAQ|cookbook on cooki...|
|"Mexican Everyday...|Whether you buy t...|A149SA6L13J3E8|whether you buy t...|
|"Mexican Everyday...|I've enjoyed watc...|A1RROHFHHCIVM7|ive enjoyed watch...|
|"Mexican Everyday...|I bought this gif...|A2O6JIFMT8VG7D|i bought this gif...|
+--------------------+--------------------+--------------+--------------------+
only showing top 5 rows



In [15]:
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType

In [16]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, MinHashLSH

In [17]:
tokenizer = Tokenizer(inputCol="clean_text", outputCol="words")
df_tokenized = tokenizer.transform(df_clean)

In [18]:
from pyspark.ml.feature import StopWordsRemover

# 1. Define custom stopwords
custom_stopwords = ["book", "read", "recipe", "good", "great"]

# Combine with default stopwords
default_stopwords = StopWordsRemover.loadDefaultStopWords("english")
all_stopwords = default_stopwords + custom_stopwords

remover = StopWordsRemover(inputCol="words", outputCol="filtered_text", stopWords=all_stopwords)
df_no_stop = remover.transform(df_tokenized)

In [19]:
# Create word-level shingles (n-grams)
def create_word_shingles(words, n=2):  
    if len(words) < n:
        return [" ".join(words)]
    return [" ".join(words[i:i+n]) for i in range(len(words) - n + 1)]

# Define UDF
shingle_udf = udf(lambda words: create_word_shingles(words, n=2), ArrayType(StringType()))

# Apply UDF to generate shingles
df_shingled = df_no_stop.withColumn("shingles", shingle_udf(col("filtered_text")))

df_shingled.show(5)

+--------------------+--------------------+--------------+--------------------+--------------------+--------------------+--------------------+
|               Title|         review/text|       User_id|          clean_text|               words|       filtered_text|            shingles|
+--------------------+--------------------+--------------+--------------------+--------------------+--------------------+--------------------+
|"Cooking with Sei...|The book was deli...|A1FTPHKCHYAJTY|the book was deli...|[the, book, was, ...|[delivered, speed...|[delivered speedy...|
|"Cooking with Sei...|Cookbook on Cooki...| AFYO0DI7QNLAQ|cookbook on cooki...|[cookbook, on, co...|[cookbook, cookin...|[cookbook cooking...|
|"Mexican Everyday...|Whether you buy t...|A149SA6L13J3E8|whether you buy t...|[whether, you, bu...|[whether, buy, fr...|[whether buy, buy...|
|"Mexican Everyday...|I've enjoyed watc...|A1RROHFHHCIVM7|ive enjoyed watch...|[ive, enjoyed, wa...|[ive, enjoyed, wa...|[ive enjoyed, enj...|

In [20]:
# Create feature vectors (using hashing trick)
from pyspark.ml.feature import HashingTF

hashing_tf = HashingTF(inputCol="shingles", outputCol="features", numFeatures=1 << 20)  # 2^20 features  #transform a sequence of terms (like words or shingles) into a fixed-length feature vector.
featurized_data = hashing_tf.transform(df_shingled)   #each bigram gets hashed to an index between 0 and 1,048,575 and the value = freq of occurence (sparce vector).

In [21]:
featurized_data.show(5)

+--------------------+--------------------+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|               Title|         review/text|       User_id|          clean_text|               words|       filtered_text|            shingles|            features|
+--------------------+--------------------+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|"Cooking with Sei...|The book was deli...|A1FTPHKCHYAJTY|the book was deli...|[the, book, was, ...|[delivered, speed...|[delivered speedy...|(1048576,[5006,39...|
|"Cooking with Sei...|Cookbook on Cooki...| AFYO0DI7QNLAQ|cookbook on cooki...|[cookbook, on, co...|[cookbook, cookin...|[cookbook cooking...|(1048576,[6129,35...|
|"Mexican Everyday...|Whether you buy t...|A149SA6L13J3E8|whether you buy t...|[whether, you, bu...|[whether, buy, fr...|[whether buy, buy...|(1048576,[73610,7...|
|"Mexican Everyd

In [22]:
from pyspark.sql.functions import monotonically_increasing_id

df_with_id = featurized_data.withColumn("Review_ID", monotonically_increasing_id())

In [23]:
# Apply MinHash
mh = MinHashLSH(inputCol="features", outputCol="hashes", numHashTables=6)
model = mh.fit(df_with_id)

In [24]:
# Find similar reviews
similar_reviews = model.approxSimilarityJoin(
    df_with_id, df_with_id, 0.8, distCol="JaccardDistance"
)

In [25]:
filtered_df = similar_reviews.filter(col("datasetA.Review_ID") != col("datasetB.Review_ID"))

In [26]:
# keeping only one direction of each pair
filtered_df = filtered_df.filter(col("datasetA.review_ID") < col("datasetB.review_ID"))

In [27]:
# exclude pairs with distance = 0
filtered_df = filtered_df.filter(col("JaccardDistance") > 0)

In [28]:
top3 = (
    filtered_df
    .orderBy(col("JaccardDistance").asc())
    .limit(3)
)

In [ ]:
top3.select(
    col("datasetA.review/text").alias("review/text_A"),
    col("datasetA.Title").alias("Title_A"),
    col("datasetB.review/text").alias("review/text_B"),
    col("datasetB.Title").alias("Title_B"),
    col("JaccardDistance")
).show(5, truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

The pipeline loads Amazon book review data, cleans and preprocesses the review text, generates 5-character shingles (n-grams), and converts these into feature vectors using the hashing trick. It then applies MinHash Locality Sensitive Hashing (LSH) to find pairs of similar reviews based on Jaccard distance. Finally, it filters out duplicate and self-pairs, sorts by similarity, and displays the top similar review pairs.

## Let's try different approach

In [30]:
from pyspark.ml.feature import Tokenizer, HashingTF, IDF, BucketedRandomProjectionLSH
from pyspark.sql.functions import col

In [31]:
from pyspark.sql.functions import size

# exclude rows where 'filtered_text' has less than 5 words
featurized_data = featurized_data.filter(size(col("filtered_text")) >= 6)

In [32]:
# Step 3: Apply IDF to get TF-IDF vectors
idf = IDF(inputCol="features", outputCol="tfidf_features")
idf_model = idf.fit(featurized_data)
df_tfidf = idf_model.transform(featurized_data)

Yes, applying TF-IDF in this context makes sense. After converting the text data into feature vectors using HashingTF (which captures term frequencies), using IDF adjusts these frequencies by reducing the weight of terms that appear frequently across many documents and increasing the weight of rarer, more distinctive terms. This results in feature vectors that better represent the unique content of each review, which is especially useful when comparing reviews for similarity. Using TF-IDF can improve the quality of similarity search and clustering compared to using raw term frequencies alone.

In [33]:
df_tfidf.show(5)

+--------------------+--------------------+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|               Title|         review/text|       User_id|          clean_text|               words|       filtered_text|            shingles|            features|      tfidf_features|
+--------------------+--------------------+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|"Cooking with Sei...|The book was deli...|A1FTPHKCHYAJTY|the book was deli...|[the, book, was, ...|[delivered, speed...|[delivered speedy...|(1048576,[5006,39...|(1048576,[5006,39...|
|"Cooking with Sei...|Cookbook on Cooki...| AFYO0DI7QNLAQ|cookbook on cooki...|[cookbook, on, co...|[cookbook, cookin...|[cookbook cooking...|(1048576,[6129,35...|(1048576,[6129,35...|
|"Mexican Everyday...|Whether you buy t...|A149SA6L13J3E8|whether you buy t

In [34]:
from pyspark.sql.functions import monotonically_increasing_id

df_with_id = df_tfidf.withColumn("Review_ID", monotonically_increasing_id())

In [35]:
# Step 4: Use LSH with Cosine-like projection (BucketedRandomProjectionLSH)
brp = BucketedRandomProjectionLSH(inputCol="features", outputCol="hashes", bucketLength=2.5, numHashTables=4)  #number of different “ways” you're looking for similarity.


brp_model = brp.fit(df_with_id)

In [36]:
# Step 5: Self-join to find similar review pairs
similar_pairs = brp_model.approxSimilarityJoin(df_with_id, df_with_id, threshold=5.0, distCol="distance")

In [37]:
# Step 6: Filter out duplicates and self-pairs
similar_pairs_filtered = similar_pairs \
    .filter(col("datasetA.Review_ID") < col("datasetB.Review_ID")) \
    .filter(col("datasetA.Review_ID") != col("datasetB.Review_ID")) \
    .select("datasetA.Title", "datasetB.Title", "datasetA.User_id", "datasetB.User_id", "datasetA.review/text", "datasetB.review/text", "distance")

In [38]:
similar_pairs_filtered.orderBy("distance").show(3, truncate=100)

+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+--------------+--------------+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+--------+
|                                                                                               Title|                                                                                               Title|       User_id|       User_id|                                                                                         review/text|                                                                                         review/text|distance|
+----------------------------------------------------------------------------------------------------+--------

In [39]:
# exlude pairs with distance = 0
similar_pairs_filtered = similar_pairs_filtered.filter(col("distance") > 0)

In [40]:
similar_pairs_filtered.orderBy("distance").show(5, truncate=100)

+----------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+--------------+--------------+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+------------------+
|                                                                                         Title|                                                                                               Title|       User_id|       User_id|                                                                                         review/text|                                                                                         review/text|          distance|
+----------------------------------------------------------------------------------------------+------